In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import dm4bem


l_1 = 7                               # m length of the room 1 
l_2 = 6                               # m length of the room 2
h = 2.5                               # m height of the room 1 and 2
w = 6                                 # m width of the rooms
S_door = 1.9                          # m² surface of inside and outside door 
Sg = 2.8                              # m² surface of the glass 
S_wall_out_1 = l_1*h - Sg - S_door    # m² surface of concrete & insulation of the outside wall room 1
S_wall_out_2 = l_2*h - Sg             # m² surface of concrete & insulation of the outside wall room 2
S_wall_in = w*h - S_door              # m² surface of insulation of the inside wall 




air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

concrete_wallout_1 = {'Conductivity': 1.400,
            'Density': 2300.0,
            'Specific heat': 880,
            'Width': 0.35,
            'Surface': S_wall_out_1}

concrete_wallout_2 = {'Conductivity': 1.400,
            'Density': 2300.0,
            'Specific heat': 880,
            'Width': 0.35,
            'Surface': S_wall_out_2}

concrete_in = {'Conductivity': 1.400,
            'Density': 2300.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': S_wall_in}

insulation_wallout_1 = {'Conductivity': 0.027,
              'Density': 55.0,
              'Specific heat': 1210,
              'Width': 0.2,
              'Surface': S_wall_out_1}

insulation_wallout_2 = {'Conductivity': 0.027,
              'Density': 55.0,
              'Specific heat': 1210,
              'Width': 0.2,
              'Surface': S_wall_out_2}

insulation_in = {'Conductivity': 0.027,
              'Density': 55.0,
              'Specific heat': 1210,
              'Width': 0.1,
              'Surface': S_wall_in}

glass = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': Sg}

door = {'Conductivity': 0.2,
         'Density': 500,
         'Specific heat': 1250,
         'Width': 0.1,
         'Surface': S_door}

wall_out_1 = pd.DataFrame.from_dict({'Layer_out': concrete_wallout_1,
                               'Layer_in': insulation_wallout_1,
                               'Door': door,
                               'Glass': glass},
                              orient='index')

wall_out_2 = pd.DataFrame.from_dict({'Layer_out': concrete_wallout_2,
                               'Layer_in': insulation_wallout_2,
                               'Glass': glass},
                              orient='index')

wall_in = pd.DataFrame.from_dict({'Layer_out': concrete_in,
                               'Layer_in': insulation_in,
                               'Door': door},
                              orient='index')

# radiative properties
ε_wLW = 0.85        # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90        # long wave emmisivity: glass pyrex
ε_wood_SW = 0.85    # short wave emmisivity: wood
α_wSW = 0.25        # short wave absortivity: white smooth surface
α_gSW = 0.38        # short wave absortivity: reflective blue glass
τ_gSW = 0.30        # short wave transmitance: reflective blue glass


σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant

# convection 
h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)



In [ ]:
#conduction
G_cd_out_1 = pd.DataFrame({'Conductance': [wall_out_1.loc['Layer_in']['Conductivity'] / wall_out_1.loc['Layer_in']['Width'] * wall_out_1.loc['Layer_in']['Surface'],
                                           wall_out_1.loc['Layer_out']['Conductivity'] / wall_out_1.loc['Layer_out']['Width'] * wall_out_1.loc['Layer_out']['Surface']]})

G_cd_out_2 = pd.DataFrame({'Conductance': [wall_out_2.loc['Layer_in']['Conductivity'] / wall_out_2.loc['Layer_in']['Width'] * wall_out_2.loc['Layer_in']['Surface'],
                                           wall_out_2.loc['Layer_out']['Conductivity'] / wall_out_2.loc['Layer_out']['Width'] * wall_out_2.loc['Layer_out']['Surface']]})


In [12]:
G_cd_in = pd.DataFrame({'Conductance': [wall_in.loc['Layer_in']['Conductivity'] / wall_in.loc['Layer_in']['Width'] * wall_in.loc['Layer_in']['Surface'],
                                           wall_in.loc['Layer_out']['Conductivity'] / wall_in.loc['Layer_out']['Width'] * wall_in.loc['Layer_out']['Surface']]})
G_cd_in

,Conductance
0,3.537
1,91.700


In [ ]:
Gc_out_1= h * wall_out_1['Surface'][0]       # wall out 1
Gd_out_1 = h * wall_out_1['Surface'][2]      # door out 1
Gg_out_1 = h * wall_out_1['Surface'][3]      # glass out 1


Gc_out_2 = h * wall_out_2['Surface'][0]     # wall out 2
Gg_out_2 = h * wall_out_2['Surface'][2]     # glass out 2


Gc_in = h.loc['h', 'in'] * wall_in['Surface'][0]           # wall in
Gd_in = h.loc['h', 'in'] * wall_in['Surface'][2]           # door in



In [ ]:
# ventilation flow rate
Va_1 = l_1*h*w                  # m³, volume of air
ACH = 1                     # air changes per hour
Va_dot_1 = ACH / 3600 * Va_1    # m³/s, air infiltration

Va_2 = l_2*h*w                  # m³, volume of air
ACH = 1                     # air changes per hour
Va_dot_2 = ACH / 3600 * Va_2    # m³/s, air infiltration


In [ ]:
# ventilation & advection
Gv_1 = air['Density'] * air['Specific heat'] * Va_dot_1
Gv_2 = air['Density'] * air['Specific heat'] * Va_dot_2

In [ ]:
# P-controler gain for room 1
Kp_1 = 1e4            # almost perfect controller Kp -> ∞
Kp_1 = 1e-3           # no controller Kp -> 0
Kp_1 = 0

# P-controler gain for room 2
Kp_2 = 1e4            # almost perfect controller Kp -> ∞
Kp_2 = 1e-3           # no controller Kp -> 0
Kp_2 = 0

In [ ]:
#Thermal Capacities Wall
C_out_1 = wall_out_1['Density'] * wall_out_1['Specific heat'] * wall_out_1['Surface'] * wall_out_1['Width']
C_out_2 = wall_out_2['Density'] * wall_out_2['Specific heat'] * wall_out_2['Surface'] * wall_out_2['Width']
C_in = wall_in['Density'] * wall_in['Specific heat'] * wall_in['Surface'] * wall_in['Width']

#C_out_1,C_out_2,C_in